In [1]:
import hail as hl
import pyspark
from hail.plot import show
hl.plot.output_notebook()

from bokeh.plotting import output_file, save, reset_output

sc = pyspark.SparkContext()
tmp_dir = "hdfs://spark-master:9820/"
lustre_dir = "file:///lustre/scratch123/qc/"
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference="GRCh38")

Loading BokehJS ...

2022-08-17 12:55:44 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-08-17 12:55:44 WARN  DependencyUtils:69 - Local jar /opt/hadoop/share/hadoop/tools/lib/aws-java-sdk-bundle-1.11.375.jar does not exist, skipping.
2022-08-17 12:55:44 INFO  SparkContext:57 - Running Spark version 3.1.2
2022-08-17 12:55:44 INFO  ResourceUtils:57 - ==============================================================
2022-08-17 12:55:44 INFO  ResourceUtils:57 - No custom resources configured for spark.driver.
2022-08-17 12:55:44 INFO  ResourceUtils:57 - ==============================================================
2022-08-17 12:55:44 INFO  SparkContext:57 - Submitted application: pyspark-shell
2022-08-17 12:55:44 INFO  ResourceProfile:57 - Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 37335, script: , vendor: , offHeap -> name: off

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jar

2022-08-17 12:55:47 INFO  Hail:28 - SparkUI: http://spark-master:4040


Running on Apache Spark version 3.1.2
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /home/ubuntu/jupyter/hail-20220817-1255-0.2.97-937922d7f46c.log


In [5]:
#pre-variant QC counts
mtdir = lustre_dir + 'matrixtables/'
broad_mtfile = mtdir + "gatk_calls_from_broad_split_varqc_multi_resequenced_samples.mt"
sanger_reseq_mtfile = mtdir + "resequenced_samples_gatk_unprocessed_split_multi_varqc.mt"
sanger_orig_mtfile = mtdir + "gatk_unprocessed_high_snp_samples_split_multi_varqc.mt"
sanger_reseq_only_mtfile = mtdir + "sanger_reseq_variants_not_in_broad.mt"
sanger_orig_only_mtfile = mtdir + "sanger_orig_variants_not_in_broad.mt"
broad_not_sanger_reseq_mtfile = mtdir + "broad_variants_not_in_sanger_reseq.mt"
broad_not_sanger_orig_mtfile = mtdir + "broad_variants_not_in_sanger_orig.mt"

In [6]:
broad_mt = hl.read_matrix_table(broad_mtfile) 
broad_mt.count()

(338808, 49)

In [8]:
sanger_reseq_mt = hl.read_matrix_table(sanger_reseq_mtfile) 
sanger_reseq_mt.count()

(325647, 49)

In [9]:
sanger_orig_mt = hl.read_matrix_table(sanger_orig_mtfile) 
sanger_orig_mt.count()

(4216339, 49)

In [11]:
broad_not_sanger_reseq_mt = hl.read_matrix_table(broad_not_sanger_reseq_mtfile) 
broad_not_sanger_reseq_mt.count()

(41903, 49)

In [12]:
broad_not_sanger_orig_mt = hl.read_matrix_table(broad_not_sanger_orig_mtfile) 
broad_not_sanger_orig_mt.count()

(38034, 49)

In [13]:
sanger_reseq_only_mt = hl.read_matrix_table(sanger_reseq_only_mtfile) 
sanger_reseq_only_mt.count()

(28742, 49)

In [14]:
sanger_orig_only_mt = hl.read_matrix_table(sanger_orig_only_mtfile) 
sanger_orig_only_mt.count()

(3915565, 49)

In [15]:
#snp and indel counts
broad_mt_snp = broad_mt.filter_rows(hl.is_snp(broad_mt.alleles[0], broad_mt.alleles[1]))
broad_mt_snp.count()

(262167, 49)

In [16]:
broad_mt_indel = broad_mt.filter_rows(hl.is_indel(broad_mt.alleles[0], broad_mt.alleles[1]))
broad_mt_indel.count()

(76640, 49)

In [18]:
sanger_reseq_mt_snp = sanger_reseq_mt.filter_rows(hl.is_snp(sanger_reseq_mt.alleles[0], sanger_reseq_mt.alleles[1]))
sanger_reseq_mt_snp.count()

(261339, 49)

In [19]:
sanger_reseq_mt_indel = sanger_reseq_mt.filter_rows(hl.is_indel(sanger_reseq_mt.alleles[0], sanger_reseq_mt.alleles[1]))
sanger_reseq_mt_indel.count()

(64308, 49)

In [20]:
sanger_orig_mt_snp = sanger_orig_mt.filter_rows(hl.is_snp(sanger_orig_mt.alleles[0], sanger_orig_mt.alleles[1]))
sanger_orig_mt_snp.count()

(4143550, 49)

In [21]:
sanger_orig_mt_indel = sanger_orig_mt.filter_rows(hl.is_indel(sanger_orig_mt.alleles[0], sanger_orig_mt.alleles[1]))
sanger_orig_mt_indel.count()

(72789, 49)

In [22]:
sanger_reseq_only_mt_snp = sanger_reseq_only_mt.filter_rows(hl.is_snp(sanger_reseq_only_mt.alleles[0], sanger_reseq_only_mt.alleles[1]))
sanger_reseq_only_mt_snp.count()

(19715, 49)

In [23]:
sanger_reseq_only_mt_indel = sanger_reseq_only_mt.filter_rows(hl.is_indel(sanger_reseq_only_mt.alleles[0], sanger_reseq_only_mt.alleles[1]))
sanger_reseq_only_mt_indel.count()

(9027, 49)

In [24]:
sanger_orig_only_mt_snp = sanger_orig_only_mt.filter_rows(hl.is_snp(sanger_orig_only_mt.alleles[0], sanger_orig_only_mt.alleles[1]))
sanger_orig_only_mt_snp.count()

(3900212, 49)

In [25]:
sanger_orig_only_mt_indel = sanger_orig_only_mt.filter_rows(hl.is_indel(sanger_orig_only_mt.alleles[0], sanger_orig_only_mt.alleles[1]))
sanger_orig_only_mt_indel.count()

(15353, 49)

In [26]:
broad_not_sanger_reseq_mt_snp = broad_not_sanger_reseq_mt.filter_rows(hl.is_snp(broad_not_sanger_reseq_mt.alleles[0], broad_not_sanger_reseq_mt.alleles[1]))
broad_not_sanger_reseq_mt_snp.count()

(20543, 49)

In [27]:
broad_not_sanger_reseq_mt_indel = broad_not_sanger_reseq_mt.filter_rows(hl.is_indel(broad_not_sanger_reseq_mt.alleles[0], broad_not_sanger_reseq_mt.alleles[1]))
broad_not_sanger_reseq_mt_indel.count()

(21359, 49)

In [28]:
broad_not_sanger_orig_mt_snp = broad_not_sanger_orig_mt.filter_rows(hl.is_snp(broad_not_sanger_orig_mt.alleles[0], broad_not_sanger_orig_mt.alleles[1]))
broad_not_sanger_orig_mt_snp.count()

(18829, 49)

In [29]:
broad_not_sanger_orig_mt_indel = broad_not_sanger_orig_mt.filter_rows(hl.is_indel(broad_not_sanger_orig_mt.alleles[0], broad_not_sanger_orig_mt.alleles[1]))
broad_not_sanger_orig_mt_indel.count()

(19204, 49)

In [45]:
#after variant QC with RF and appropriate cut offs
sanger_orig_filtered_mtfile_rf_pass = mtdir + "gatk_calls_from_sanger_orig_samples_in_broad_after_rf.mt"
sanger_orig_only_mtfile_rf_pass = mtdir + "sanger_orig_variants_not_in_broad_after_rf.mt"
broad_not_sanger_orig_mtfile_rf_pass = mtdir + "broad_variants_not_in_sanger_orig_after_rf.mt"

sanger_reseq_filtered_mtfile_rf_pass = mtdir + "gatk_calls_from_sanger_reseq_samples_in_broad_after_rf.mt"
sanger_reseq_only_mtfile_rf_pass = mtdir + "sanger_reseq_variants_not_in_broad_after_rf.mt"
broad_not_sanger_reseq_mtfile_rf_pass = mtdir + "broad_variants_not_in_sanger_reseq_after_rf.mt"

In [46]:
sanger_reseq_mt_qc = hl.read_matrix_table(sanger_reseq_filtered_mtfile_rf_pass) 
sanger_reseq_mt_qc.count()

(282658, 49)

In [47]:
sanger_orig_mt_qc = hl.read_matrix_table(sanger_orig_filtered_mtfile_rf_pass) 
sanger_orig_mt_qc.count()

(521917, 49)

In [48]:
broad_not_sanger_orig_qc_mt = hl.read_matrix_table(broad_not_sanger_orig_mtfile_rf_pass) 
broad_not_sanger_orig_qc_mt.count()

(57215, 49)

In [49]:
broad_not_sanger_reseq_qc_mt = hl.read_matrix_table(broad_not_sanger_reseq_mtfile_rf_pass) 
broad_not_sanger_reseq_qc_mt.count()

(66102, 49)

In [50]:
sanger_reseq_mt_qc_only = hl.read_matrix_table(sanger_reseq_only_mtfile_rf_pass) 
sanger_reseq_mt_qc_only.count()

(9952, 49)

In [51]:
sanger_orig_mt_qc_only = hl.read_matrix_table(sanger_orig_only_mtfile_rf_pass) 
sanger_orig_mt_qc_only.count()

(240324, 49)

In [53]:
sanger_reseq_mt_qc_snp = sanger_reseq_mt_qc.filter_rows(hl.is_snp(sanger_reseq_mt_qc.alleles[0], sanger_reseq_mt_qc.alleles[1]))
sanger_reseq_mt_qc_snp.count()

(228868, 49)

In [54]:
sanger_reseq_mt_qc_indel = sanger_reseq_mt_qc.filter_rows(hl.is_indel(sanger_reseq_mt_qc.alleles[0], sanger_reseq_mt_qc.alleles[1]))
sanger_reseq_mt_qc_indel.count()

(53790, 49)

In [55]:
sanger_orig_mt_qc_snp = sanger_orig_mt_qc.filter_rows(hl.is_snp(sanger_orig_mt_qc.alleles[0], sanger_orig_mt_qc.alleles[1]))
sanger_orig_mt_qc_snp.count()

(470791, 49)

In [56]:
sanger_orig_mt_qc_indel = sanger_orig_mt_qc.filter_rows(hl.is_indel(sanger_orig_mt_qc.alleles[0], sanger_orig_mt_qc.alleles[1]))
sanger_orig_mt_qc_indel.count()

(51126, 49)

In [57]:
broad_not_sanger_reseq_qc_snp = broad_not_sanger_reseq_qc_mt.filter_rows(hl.is_snp(broad_not_sanger_reseq_qc_mt.alleles[0], broad_not_sanger_reseq_qc_mt.alleles[1]))
broad_not_sanger_reseq_qc_snp.count()

(37548, 49)

In [58]:
broad_not_sanger_reseq_qc_indel = broad_not_sanger_reseq_qc_mt.filter_rows(hl.is_indel(broad_not_sanger_reseq_qc_mt.alleles[0], broad_not_sanger_reseq_qc_mt.alleles[1]))
broad_not_sanger_reseq_qc_indel.count()

(28553, 49)

In [59]:
broad_not_sanger_orig_qc_snp = broad_not_sanger_orig_qc_mt.filter_rows(hl.is_snp(broad_not_sanger_orig_qc_mt.alleles[0], broad_not_sanger_orig_qc_mt.alleles[1]))
broad_not_sanger_orig_qc_snp.count()

(25813, 49)

In [60]:
broad_not_sanger_orig_qc_indel = broad_not_sanger_orig_qc_mt.filter_rows(hl.is_indel(broad_not_sanger_orig_qc_mt.alleles[0], broad_not_sanger_orig_qc_mt.alleles[1]))
broad_not_sanger_orig_qc_indel.count()

(31401, 49)

In [61]:
sanger_reseq_mt_qc_only_snp = sanger_reseq_mt_qc_only.filter_rows(hl.is_snp(sanger_reseq_mt_qc_only.alleles[0], sanger_reseq_mt_qc_only.alleles[1]))
sanger_reseq_mt_qc_only_snp.count()

(4249, 49)

In [62]:
sanger_reseq_mt_qc_only_indel = sanger_reseq_mt_qc_only.filter_rows(hl.is_indel(sanger_reseq_mt_qc_only.alleles[0], sanger_reseq_mt_qc_only.alleles[1]))
sanger_reseq_mt_qc_only_indel.count()

(5703, 49)

In [63]:
sanger_orig_mt_qc_only_snp = sanger_orig_mt_qc_only.filter_rows(hl.is_snp(sanger_orig_mt_qc_only.alleles[0], sanger_orig_mt_qc_only.alleles[1]))
sanger_orig_mt_qc_only_snp.count()

(234437, 49)

In [64]:
sanger_orig_mt_qc_only_indel = sanger_orig_mt_qc_only.filter_rows(hl.is_indel(sanger_orig_mt_qc_only.alleles[0], sanger_orig_mt_qc_only.alleles[1]))
sanger_orig_mt_qc_only_indel.count()

(5887, 49)